In [526]:
import requests
import pandas as pd
import json
from dateutil import parser


In [527]:
API_KEY = "a2faf0838c2aacce44be67cea8b40c06-48d69bac26199a4d5850294881134c34"
ACCOUNT_ID = "101-004-21509763-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [528]:
session = requests.Session()

In [529]:
session.headers.update({
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {API_KEY}'
})

In [530]:
params = dict(
    count = 10,
    granularity = "H1"
)

In [531]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [532]:
response = session.get(url, params=None, data=None, headers=None)

In [533]:
response.status_code

200

In [534]:
data = response.json()

In [535]:
data.keys()

dict_keys(['instruments', 'lastTransactionID'])

In [536]:
instruments_list = data["instruments"]

In [537]:
len(instruments_list)

123

In [538]:
instruments_list[1].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [539]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [540]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i }

In [541]:
instruments_dict['AU200_AUD']

{'name': 'AU200_AUD',
 'type': 'CFD',
 'displayName': 'Australia 200',
 'pipLocation': 0,
 'displayPrecision': 1,
 'tradeUnitsPrecision': 1,
 'marginRate': '0.05'}

In [542]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [543]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
    

In [544]:

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
            
        final_data.append(new_dict)
    df = pd.DataFrame(final_data)
    return df


In [545]:
code, data = fetch_candles('EUR_USD', count=20, granularity="H4")

In [546]:
code

200

In [547]:
data[0]

{'complete': True,
 'volume': 38457,
 'time': '2023-11-03T13:00:00.000000000Z',
 'bid': {'o': '1.07088', 'h': '1.07387', 'l': '1.06928', 'c': '1.07386'},
 'mid': {'o': '1.07095', 'h': '1.07394', 'l': '1.06936', 'c': '1.07394'},
 'ask': {'o': '1.07102', 'h': '1.07402', 'l': '1.06943', 'c': '1.07402'}}

In [548]:
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']
for p in prices:
    for o in ohlc:
        print(f"{p}_{o}")

mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [549]:
get_candles_df(data)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-11-03 13:00:00+00:00,38457,1.07095,1.07394,1.06936,1.07394,1.07088,1.07387,1.06928,1.07386,1.07102,1.07402,1.06943,1.07402
1,2023-11-03 17:00:00+00:00,11059,1.07394,1.07469,1.07224,1.07286,1.07386,1.07461,1.07206,1.07269,1.07401,1.07477,1.07232,1.07303
